In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [92]:
X = train_data.drop('SalePrice',axis = 1)
Y = train_data.SalePrice
Xt = test_data
numeric_cols =test_data.columns[test_data.dtypes != 'object'].values
categorical_cols = list(set(X.columns.values.tolist()) - set(numeric_cols))

In [95]:
X_num = X[numeric_cols]
X_cat = X[categorical_cols]
X_num.LotFrontage.fillna(value = X_num.LotFrontage.mean(),inplace=True)
X_num.MasVnrArea.fillna(value = X_num.MasVnrArea.median(),inplace=True)
X_num.GarageYrBlt.fillna(value = X_num.GarageYrBlt.median(),inplace=True)
X_cat.replace(np.nan,'NA',inplace=True)

Xt_num = Xt[numeric_cols]
Xt_cat = Xt[categorical_cols]
Xt_num.LotFrontage.fillna(value = Xt_num.LotFrontage.mean(),inplace=True)
Xt_num.MasVnrArea.fillna(value = Xt_num.MasVnrArea.median(),inplace=True)
Xt_num.BsmtFinSF1.fillna(value = Xt_num.BsmtFinSF1.median(),inplace=True)
Xt_num.BsmtFinSF2.fillna(value = Xt_num.BsmtFinSF2.median(),inplace=True)
Xt_num.BsmtUnfSF.fillna(value = Xt_num.BsmtUnfSF.median(),inplace=True)
Xt_num.TotalBsmtSF.fillna(value = Xt_num.TotalBsmtSF.median(),inplace=True)
Xt_num.BsmtFullBath.fillna(value = Xt_num.BsmtFullBath.median(),inplace=True)
Xt_num.BsmtHalfBath.fillna(value = Xt_num.BsmtHalfBath.median(),inplace=True)
Xt_num.GarageYrBlt.fillna(value = Xt_num.GarageYrBlt.median(),inplace=True)
Xt_num.GarageCars.fillna(value = Xt_num.GarageCars.median(),inplace=True)
Xt_num.GarageArea.fillna(value = Xt_num.GarageArea.median(),inplace=True)
Xt_cat.replace(np.nan,'NA',inplace=True)

C:\Anaconda3\lib\site-packages\pandas\core\generic.py:5430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
C:\Anaconda3\lib\site-packages\pandas\core\frame.py:3795: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)


In [6]:
for i in X_cat:
    X_cat[i] = X_cat[i].apply(str)
for j in Xt_cat:
    Xt_cat[j] = Xt_cat[j].apply(str)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


Мы преобразуем категориальный признак при помощи бинарного кода: каждой категории ставим в соответствие набор из нулей и единиц

In [7]:
from sklearn.feature_extraction import DictVectorizer as DV

encoder = DV(sparse=False)
X_cat_oh = encoder.fit_transform(X_cat.T.to_dict().values())
Xt_cat_oh = encoder.transform(Xt_cat.T.to_dict().values())

In [8]:
from sklearn import cross_validation, linear_model, metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

C:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [9]:
(X_cat_train, X_cat_test,  y_train, y_test) = cross_validation.train_test_split(X_cat_oh, Y, 
                                     test_size=0.3, 
                                     random_state=0)
(X_num_train, X_num_test) = cross_validation.train_test_split(X_num, test_size=0.3, random_state=0)

In [17]:
# Признаки для обучения методом кроссвалидации
X_cv = np.hstack((X_num,X_cat_oh))

In [52]:
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn import svm
from sklearn.metrics import make_scorer

In [53]:
def rmsle_score(train, pred):
    rmsle_score = (np.sum((np.log1p(train)-np.log1p(pred))**2)/len(pred))**0.5
    return rmsle_score

In [57]:
def test_model(model, X_train, y_train):
    rmsle  = make_scorer(rmsle_score)
    cv = KFold(n_splits=5,shuffle=True,random_state=45)
    rmsle_score_cv  = cross_val_score(model, X_train, y_train, cv = cv ,scoring = rmsle)
    score=[rmsle_score_cv .mean()]
    
    return score

In [79]:
def fit_models(X_train, y_train):
    results={}
    
    model = linear_model.LinearRegression()
    results["Linear"]=test_model(model, X_train, y_train)
    
    model = linear_model.Ridge()
    results["Ridge"]=test_model(model, X_train, y_train)
    
    model = linear_model.BayesianRidge()
    results["Bayesian Ridge"]=test_model(model, X_train, y_train)
    
    model = linear_model.HuberRegressor()
    results["Hubber"]=test_model(model, X_train, y_train)
    
    model = linear_model.Lasso(alpha=1e-4)
    results["Lasso"]=test_model(model, X_train, y_train)
    
    model = BaggingRegressor()
    results["Bagging"]=test_model(model, X_train, y_train)
    
    model = AdaBoostRegressor()
    results["AdaBoost"]=test_model(model, X_train, y_train)
    
    model = svm.SVR()
    results["SVM RBF"]=test_model(model, X_train, y_train)
    
    model = svm.SVR(kernel="linear")
    results["SVM Linear"]=test_model(model, X_train, y_train)
    
    results = pd.DataFrame.from_dict(results,orient='index')
    results.columns=["Square Score"] 
    results = results.sort_values("Square Score", ascending=True)
    
    return results

In [80]:
fit_models(X_cv, Y)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log1p
  
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log1p
  
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log1p
  
C:\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might

,Square Score
Bagging,0.154394
Bayesian Ridge,0.156863
Ridge,0.161101
Lasso,0.176130
Linear,0.191390
SVM Linear,0.194121
AdaBoost,0.198565
Hubber,0.215247
SVM RBF,0.399660


In [65]:
# Признаки для обучения после единственного разбиения методом train_test_split
X_un_train = np.hstack((X_num_train,X_cat_train))  #обучающая выборка численные + категориальные признаки
X_un_test = np.hstack((X_num_test,X_cat_test))     #тестовая выборка для оценки качества обучения
Xt_un_test = np.hstack((Xt_num,Xt_cat_oh))         #целевая выборка, для которой необходимо построить предсказания

model = BaggingRegressor()
model.fit(X_un_train, y_train)
Y_un_test = model.predict(X_un_test)
rmsle_score(Y_un_test,y_test)
mean_squared_error(Y_un_test,y_test)

1047894776.9176257

In [78]:
real_predictions = model.predict(Xt_un_test)
submission = pd.DataFrame({'Id':test_data.Id.values,'SalePrice':real_predictions})
print(submission)
submission.to_csv("submission.csv",index=False)

        Id  SalePrice
0     1461   131390.0
1     1462   158465.0
2     1463   180100.0
3     1464   183275.0
4     1465   216692.6
5     1466   192900.0
6     1467   160810.0
7     1468   178469.0
8     1469   182219.5
9     1470   123325.0
10    1471   188259.0
11    1472    92740.0
12    1473   102550.0
13    1474   161525.0
14    1475   147603.5
15    1476   369270.5
16    1477   258822.6
17    1478   296749.3
18    1479   292196.3
19    1480   504345.1
20    1481   276626.3
21    1482   204600.0
22    1483   173255.4
23    1484   174982.5
24    1485   173643.5
25    1486   184700.0
26    1487   263550.0
27    1488   232739.8
28    1489   204208.0
29    1490   181310.0
...    ...        ...
1429  2890    93270.0
1430  2891   146550.0
1431  2892    69160.0
1432  2893   117500.0
1433  2894    61790.0
1434  2895   316438.7
1435  2896   325865.2
1436  2897   203090.0
1437  2898   155016.4
1438  2899   232518.0
1439  2900   151679.0
1440  2901   183280.0
1441  2902   188386.4
1442  2903